In [ ]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env

m = wt.watertap_setup(dynamic = False)


wt.case_study_trains.train = {"case_study": "tampa_bay",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "tampa_bay", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "seawater"}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.


In [ ]:
wt.display.show_train2(model_name=m)

In [ ]:
# RUN MODEL with optimal ro so that the model solves. Then runs again with set pressure. 

In [ ]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("unfixing feed presure and area for", key)
        
wt.run_water_tap(m = m, objective=True, skip_small=True)

# prints RO results
print("optimal ro area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print("setting feed presure for", key, "to -->", m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])

wt.run_water_tap(m = m, objective=True, print_model_results="summary")

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

In [ ]:
# If you need the system recovery to match better.... set a maximum recovery rate.

In [ ]:
from pyomo.environ import Constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.55) # THIS IS FOR TAMPA BAY
#m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.50) # THIS IS FOR SANTA BARBARA

In [ ]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 0.75 # 0.75 for Tampa Bay

In [ ]:
wt.run_water_tap(m = m, objective=True, print_model_results="summary")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

In [ ]:
# RESET AREA BASED ON RO RECOVERY
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [ ]:
# Readjust recovery constraint and deactivate objective constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery >= 0)
m.fs.objective_function.deactivate() # deactivate LCOW objective function

In [ ]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m = m, objective=False, print_model_results="summary")

In [ ]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m = m, case_study = wt.case_study_trains.source_water["case_study"], 
                                scenario = wt.case_study_trains.source_water["scenario"])

In [ ]:
###### SENSITIVITY ANALYSES 

In [ ]:
# Find bounds for RO area:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).membrane_area[0])
        getattr(m.fs, key).membrane_area.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0] * 1, sense=env.minimize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).membrane_area[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).membrane_area[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).membrane_area[0]())

        getattr(m.fs, key).membrane_area.fix(stash_value)


In [ ]:
# Find bounds for RO pessure:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
        stash_value = value(getattr(m.fs, key).feed.pressure[0])
        getattr(m.fs, key).feed.pressure.unfix()
        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.minimize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Minimum -->", getattr(m.fs, key).feed.pressure[0]())

        m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.maximize)
        wt.run_water_tap(m = m, objective=False)
        print("LCOW -->", m.fs.costing.LCOW())
        print(key, "Maximum -->", getattr(m.fs, key).feed.pressure[0]())

        getattr(m.fs, key).feed.pressure.fix(stash_value)

In [ ]:
m.fs.objective_function1.deactivate()

In [ ]:
# run to rest before sensitivity
wt.run_water_tap(m = m, objective=False)
print("LCOW -->", m.fs.costing.LCOW())